# softmax回归的简洁实现

我们在[“线性回归的简洁实现”](linear-regression-gluon.ipynb)一节中已经了解了使用Gluon实现模型的便利。下面，让我们再次使用Gluon来实现一个softmax回归模型。首先导入所需的包或模块。

In [17]:
%matplotlib inline
import d2lzh as d2l
from mxnet import gluon, init, autograd
from mxnet.gluon import loss as gloss, nn

## 获取和读取数据

我们仍然使用Fashion-MNIST数据集和上一节中设置的批量大小。

In [2]:
batch_size = 1
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

## 定义和初始化模型

在[“softmax回归”](softmax-regression.ipynb)一节中提到，softmax回归的输出层是一个全连接层。因此，我们添加一个输出个数为10的全连接层。我们使用均值为0、标准差为0.01的正态分布随机初始化模型的权重参数。

In [34]:
net = nn.Sequential()
net.add(nn.Dense(1000))
net.add(nn.Dense(1000))
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))

## softmax和交叉熵损失函数

如果做了上一节的练习，那么你可能意识到了分开定义softmax运算和交叉熵损失函数可能会造成数值不稳定。因此，Gluon提供了一个包括softmax运算和交叉熵损失计算的函数。它的数值稳定性更好。

In [37]:
loss = gloss.SoftmaxCrossEntropyLoss()

## 定义优化算法

我们使用学习率为0.1的小批量随机梯度下降作为优化算法。

In [38]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

## 训练模型

接下来，我们使用上一节中定义的训练函数来训练模型。

In [44]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, trainer=None):
    start = time()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            if trainer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                trainer.step(batch_size)  # “softmax回归的简洁实现”一节将用到
            y = y.astype('float32')
            print(time()-start)
            train_l_sum += l.asscalar()
            print('----------',time()-start)
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [45]:
from time import time
num_epochs = 5000
train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, trainer)

0.004956483840942383
---------- 0.016921520233154297
0.019913911819458008
---------- 0.027890443801879883
0.030883073806762695
---------- 0.040856361389160156
0.04384803771972656
---------- 0.05182671546936035
0.054819345474243164
---------- 0.06180095672607422
0.06479263305664062
---------- 0.07277131080627441
0.07576322555541992
---------- 0.08274435997009277
0.08573698997497559
---------- 0.09371495246887207
0.09670734405517578
---------- 0.10368800163269043
0.10668110847473145
---------- 0.11366105079650879
0.1166536808013916
---------- 0.12662649154663086
0.12961983680725098
---------- 0.13759732246398926
0.13959240913391113
---------- 0.14657330513000488
0.1495664119720459
---------- 0.15754413604736328
0.15953898429870605
---------- 0.16751718521118164
0.17052173614501953
---------- 0.1774909496307373
0.18148159980773926
---------- 0.19045686721801758
0.19245147705078125
---------- 0.20043063163757324
0.20342230796813965
---------- 0.21140050888061523
0.2133946418762207
--------

---------- 1.8732714653015137
1.8762643337249756
---------- 1.885239601135254
1.8882322311401367
---------- 1.899202585220337
1.903193473815918
---------- 1.9121668338775635
1.9151611328125
---------- 1.923137903213501
1.926131248474121
---------- 1.9351060390472412
1.9371013641357422
---------- 1.9450790882110596
1.9480717182159424
---------- 1.957047462463379
1.9590415954589844
---------- 1.9680185317993164
1.9700126647949219
---------- 1.978999137878418
1.9809839725494385
---------- 1.9899587631225586
1.9929533004760742
---------- 2.0009310245513916
2.0029256343841553
---------- 2.0119011402130127
2.014892816543579
---------- 2.022871255874634
2.0258634090423584
---------- 2.0328450202941895
2.035837173461914
---------- 2.044177770614624
2.0461723804473877
---------- 2.0541510581970215
2.0581417083740234
---------- 2.0661189556121826
2.069112539291382
---------- 2.0760927200317383
2.079085350036621
---------- 2.0880610942840576
2.0920498371124268
---------- 2.1000313758850098
2.1030

KeyboardInterrupt: 

## 小结

* Gluon提供的函数往往具有更好的数值稳定性。
* 可以使用Gluon更简洁地实现softmax回归。

## 练习

* 尝试调一调超参数，如批量大小、迭代周期和学习率，看看结果会怎样。



## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/740)

![](../img/qr_softmax-regression-gluon.svg)